In [2]:
import backtrader as bt
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


In [77]:
class VWAPStrategy(bt.Strategy):
    params = (
        ("ema_period", 50),     # EMA Period
        ('atr_period', 9) ,      # ATR Period
        ('atr_multiplier', 1.5), # ATR multiplier for SL calculation
        ("stop_loss", 0.02),  # 2% stop loss
        ("risk", 0.01),  # 1% risk
        ("pivot_window", 5),  # 5 bars for pivot high/low
        ("rrr", 2),  # Risk-Reward Ratio
        ("trailpercent", 0.05),     # Percent Price for Trailing stop
        ("sl_pips", 50),
        ("pip_value", 0.01)     # Pip value for Gold (XAUUSD)
    )
    def __init__(self):
        # initialize the EMA indicator on data feed
        # self.ema = bt.talib.EMA(self.data.close, self.params.ema_period)
        self.ema = bt.indicators.ExponentialMovingAverage(period=self.params.ema_period)
        self.atr = bt.indicators.AverageTrueRange(period=self.params.atr_period)
        self.crossover = bt.indicators.CrossOver(self.data.close, self.ema)
        self.order = None
        self.prev_pivot_high = self.data.low[0]
        self.prev_pivot_low = self.data.high[0]
        
    def _isBulluish(self, idx:int):
        if len(self) < idx:
            self.log("[Error]: Index out of bounds for `isBullish`")
        return self.data.close[idx] > self.data.open[idx]

    def _isEngulfing(self, idx:int):
        if len(self) < idx:
            self.log("[Error]: Index out of bounds for `isEnglufing`")
        return self.data.open[idx+1] < self.data.close[idx] if self._isBulluish(idx) \
            else self.data.open[idx+1] > self.data.close[idx]

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
        self.order = None
    
    def log(self, txt):
        dt = self.data.datetime.date(0)
        print(f'{dt.isoformat()} {txt}')
    
    def _calculate_size(self, price, sl_price):
        # Calculate the size of the position based on the risk percentage and stop loss
        size = (self.broker.getvalue() * self.params.risk) / abs(price - sl_price)
        return size
    
    def _update_pivots(self):
        if np.max(self.data.high.get(size=self.params.pivot_window * 2 + 1)) == self.data.high[self.params.pivot_window]:
            self.prev_pivot_high = self.data.high[self.params.pivot_window]   
        if np.min(self.data.low.get(size=self.params.pivot_window * 2 + 1)) == self.data.low[self.params.pivot_window]:
            self.prev_pivot_low = self.data.low[self.params.pivot_window]      

    def next(self):
        # if not in a market position, check for crossover from below
        if self.position:
            return
        if self.crossover[0] > 0 and (self._isEngulfing(0) or self._isEngulfing(1)):
            price = self.data.close[0]
            # sl_price = self.data.low[1]
            # sl_price = price * (1 - self.params.stop_loss)
            # sl_price = price - self.params.sl_pips * self.params.pip_value
            sl_price = price - self.atr * self.params.atr_multiplier
            tp_price = price + np.abs(sl_price - price) * self.params.rrr
            if tp_price < price or tp_price < 0:
                self.log(f'Take profit price {tp_price:.2f} is less than entry price {price:.2f}, not placing order.')
                return
            if sl_price > price or sl_price < 0:
                self.log(f'Stop loss price {sl_price:.2f} is greater than entry price {price:.2f}, not placing order.')
                return

            # size = self._calculate_size(price, sl_price)
            self.order = self.buy_bracket(
                exectype=bt.Order.Market, 
                # size=size,
                price=price,
                stopprice=sl_price,
                limitprice=tp_price,
                stopargs=dict(exectype=bt.Order.Stop),
                limitargs=dict(exectype=bt.Order.Limit),
                # trailpercent=self.params.trailpercent
            )
            self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")

        # if in a position, check for crossover from above for exit
        elif self.crossover[0] < 0 and (self._isEngulfing(0) or self._isEngulfing(1)):
            price = self.data.close[0]
            # sl_price = self.data.high[1]
            # sl_price = price * (1 + self.params.stop_loss)
            # sl_price = price + self.params.sl_pips * self.params.pip_value
            sl_price = price + self.atr * self.params.atr_multiplier
            tp_price = price - np.abs(sl_price - price) * self.params.rrr
            if tp_price > price or tp_price < 0:
                self.log(f'Take profit price {tp_price:.2f} is greater than entry price {price:.2f}, not placing order.')
                return
            if sl_price < price or sl_price < 0:
                self.log(f'Stop loss price {sl_price:.2f} is less than entry price {price:.2f}, not placing order.')
                return
            # size = self._calculate_size(price, sl_price)
            self.order = self.sell_bracket(
                exectype=bt.Order.Market, 
                # size=size,
                price=price,
                stopprice=sl_price,
                limitprice=tp_price,
                stopargs=dict(exectype=bt.Order.Stop),
                limitargs=dict(exectype=bt.Order.Limit),
                # trailpercent=self.params.trailpercent
            )
            self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")        


## Download Data from YF

In [78]:
# Download SPX500 data from Yahoo Finance
yf_params = {'start': '2018-01-01', 'end': '2024-01-01', 'interval': '1d', 'multi_level_index': False}
# data = yf.download(tickers="^GSPC", **yf_params)
data = yf.download(tickers="DJIA", **yf_params)
# data = yf.download(tickers="^VIX", **yf_params)
# data = yf.download(tickers="^RUT", **yf_params)
# data = yf.download(tickers="^IXIC", **yf_params)
# data = yf.download(tickers="GOOG", **yf_params)
# data = yf.download(tickers="AAPL", **yf_params)
# data = yf.download(tickers="MSFT", **yf_params)
# data = yf.download(tickers="AMZN", **yf_params)
# data = yf.download(tickers="TSLA", **yf_params)
# data = yf.download(tickers="META", **yf_params)
# data = yf.download(tickers="NFLX", **yf_params)
# data = yf.download(tickers="NVDA", **yf_params)
# data = yf.download(tickers="AMD", **yf_params)
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)

# data

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['DJIA']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


## Source data locally

In [79]:
data_local = pd.read_csv("../data/xauusd_1h.csv", index_col=None)
data_local['datetime'] = pd.to_datetime(data_local['datetime'])
print(data_local.head())

data_feed_local = bt.feeds.PandasData(dataname=data_local, name="xau",
                                      datetime=0, openinterest=-1,
                                      open=1,
                                      high=2,
                                      low=3,
                                      close=4,
                                      volume=5)

             datetime         open         high          low        close  \
0 2010-01-03 22:00:00  1097.880005  1097.880005  1097.449951  1097.449951   
1 2010-01-03 23:00:00  1097.680054  1100.599976  1095.979980  1095.979980   
2 2010-01-04 00:00:00  1096.010010  1096.869995  1093.449951  1094.839966   
3 2010-01-04 01:00:00  1094.869995  1095.959961  1094.239990  1095.699951   
4 2010-01-04 02:00:00  1095.670044  1099.150024  1095.630005  1098.329956   

   volume  
0       2  
1     919  
2    1596  
3     869  
4    1054  


In [80]:
cerebro = bt.Cerebro()
# cerebro.optstrategy(VWAPStrategy, 
#                     ema_period=range(30, 50, 5),
#                     sl_pips=range(50,600, 50),
#                     pip_value=100.0,
#                     rrr=np.arange(1.0, 2.0, .2)
#                     )

cerebro.addstrategy(VWAPStrategy, 
                    ema_period=50,
                    atr_period=21,
                    atr_multiplier=1.5,
                    sl_pips=200,
                    pip_value=.10,
                    rrr=2,
                    )


cerebro.broker.setcash(100_000.0)
cerebro.broker.setcommission(commission=0.001)
data_feed = bt.feeds.pandafeed.PandasData(
    dataname=data,
    datetime=0,
    openinterest=-1,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
)
cerebro.adddata(data_feed_local)

In [81]:
cerebro.addsizer(bt.sizers.PercentSizer, percents=5)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.01)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
cerebro.addanalyzer(bt.analyzers.Returns)

results = cerebro.run()

2010-01-08 Bracket order placed: Entry 1137.89, SL 1132.11, TP 1149.45
2010-01-08 BUY EXECUTED, 1138.00
2010-01-08 SELL EXECUTED, 1132.11
2010-01-08 Bracket order placed: Entry 1127.60, SL 1133.95, TP 1114.90
2010-01-08 SELL EXECUTED, 1127.64
2010-01-08 BUY EXECUTED, 1133.95
2010-01-12 Bracket order placed: Entry 1147.84, SL 1153.30, TP 1136.91
2010-01-12 SELL EXECUTED, 1147.89
2010-01-12 BUY EXECUTED, 1153.30
2010-01-12 Bracket order placed: Entry 1147.14, SL 1153.10, TP 1135.23
2010-01-12 SELL EXECUTED, 1147.07
2010-01-12 BUY EXECUTED, 1135.23
2010-01-13 Bracket order placed: Entry 1137.59, SL 1130.39, TP 1152.00
2010-01-13 BUY EXECUTED, 1137.56
2010-01-15 SELL EXECUTED, 1130.39
2010-01-18 Bracket order placed: Entry 1135.92, SL 1131.07, TP 1145.62
2010-01-18 BUY EXECUTED, 1135.85
2010-01-19 SELL EXECUTED, 1131.07
2010-01-19 Bracket order placed: Entry 1139.74, SL 1134.81, TP 1149.62
2010-01-19 BUY EXECUTED, 1139.71
2010-01-20 SELL EXECUTED, 1134.81
2010-01-25 Bracket order placed: E

In [82]:
strat = results[0]
sharpe_ratio = strat.analyzers.sharperatio.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
trade_analysis = strat.analyzers.tradeanalyzer.get_analysis()
returns = strat.analyzers.returns.get_analysis()
print(f'Sharpe Ratio: {sharpe_ratio}')
print(f'Max Drawdown: {drawdown["max"]["drawdown"]}')
print(f'Number of Trades: {trade_analysis.total.closed}')
print(f'Winning Trades: {trade_analysis.won.total}')
print(f'Losing Trades: {trade_analysis.lost.total}')
print(f'Average Trade Return: {returns["rnorm"]}')
print(f'Total Returns: {returns["rtot"]}')

Sharpe Ratio: OrderedDict([('sharperatio', -8.827087906792693)])
Max Drawdown: 17.374789601797893
Number of Trades: 2099
Winning Trades: 737
Losing Trades: 1362
Average Trade Return: -0.01027322952382351
Total Returns: -0.19070990116731343


In [73]:
%matplotlib inline
fig = cerebro.plot(iplot=True)
# plt.show()
fig

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 6 Axes>]]